In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#read_record_df = pd.read_csv("to_read.csv")
read_record_df = pd.read_csv("ratings.csv")

In [3]:
read_record_df.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [4]:
userNo,bookNo = read_record_df["user_id"].max()+1,read_record_df["book_id"].max()+1

In [5]:
userNo,bookNo

(53425, 10001)

In [6]:
record = np.zeros((userNo,bookNo),dtype=np.int)

In [7]:
for index,row in read_record_df.iterrows():
    record[int(row["user_id"])][int(row["book_id"])] = 1

# MemoryError 缩小数据集

In [8]:
record = record[0:1000,:]

In [9]:
record.shape

(1000, 10001)

In [10]:
userNo = 1000

In [11]:
f = 50
lamda = 0.01

In [12]:
def createLFM(userNo,bookNo,f):
    LFM_P = tf.Variable(tf.random_normal([userNo,f],stddev=1/(f**0.5),dtype=tf.float32))
    LFM_Q = tf.Variable(tf.random_normal([bookNo,f],stddev=1/(f**0.5),dtype=tf.float32))
    LFM_predict = tf.matmul(LFM_P,LFM_Q,transpose_b=True)
    LFM_loss = tf.reduce_sum((record-LFM_predict)**2) + lamda*((tf.reduce_sum(LFM_P)**2)+(tf.reduce_sum(LFM_Q)**2))
    return LFM_loss,LFM_predict,LFM_Q

In [13]:
loss,predict,LFM_Q = createLFM(userNo,bookNo,f)

In [14]:
learning_rate = 1e-4
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train = optimizer.minimize(loss)

In [15]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [16]:
for i in range(1,501): 
    if i % 100 == 0:
        print("迭代次数：",i)
    sess.run(train)

迭代次数： 100
迭代次数： 200
迭代次数： 300
迭代次数： 400
迭代次数： 500


In [17]:
bookLatent = sess.run(LFM_Q)

In [18]:
bookLatent

array([[-0.16923681, -0.00158055, -0.08878354, ...,  0.04679792,
        -0.05536512,  0.11234671],
       [-0.18979117, -0.09875041, -0.12413648, ...,  0.6959208 ,
         0.10727397, -0.31251496],
       [-0.32645568, -0.15352108, -0.33955187, ...,  0.15034854,
         0.07907347, -0.11648104],
       ...,
       [ 0.16430691, -0.00420873, -0.02066747, ..., -0.01850194,
        -0.15815522, -0.02967418],
       [ 0.20275553, -0.15649371, -0.0626756 , ..., -0.01190972,
        -0.06127517,  0.08397022],
       [-0.04425759,  0.00271949, -0.06645397, ...,  0.0999576 ,
         0.11564423, -0.02019908]], dtype=float32)

In [19]:
bookLatent.shape

(10001, 50)

In [20]:
latentSort = np.argsort(-bookLatent,axis=0) 

In [21]:
latentSort

array([[3743, 2775, 2411, ...,    1, 5919,  259],
       [8570, 4430, 6413, ..., 7640, 5744, 3749],
       [4018, 4400, 1412, ..., 4972,   21, 5252],
       ...,
       [   4,  104,   58, ..., 6462, 5377, 1873],
       [  11, 4561,   45, ..., 1303, 3998, 7903],
       [  31,   27,    5, ..., 2509, 1564,  104]], dtype=int64)

In [22]:
bookLatent[6540][0],bookLatent[2049][0],bookLatent[1257][0],bookLatent[2346][0],bookLatent[4503][0],bookLatent[9944][0]

(0.021487493, -0.19724533, -0.06045882, 0.24410562, -0.047746733, -0.033857204)

In [23]:
maxWightBook = latentSort[0:5,:]

In [24]:
maxWightBook

array([[3743, 2775, 2411, 7237, 1205, 2462, 6298, 4179, 2018, 8146,    8,
        9785, 3077, 4856,  117,  321,   94,   23,  463,    4, 1926,  378,
        3606, 4791,    9,  543,  592,  121, 9240, 4978,   45, 1317, 2792,
        2777, 9498,    4,   18, 5153, 1764, 5578, 4020,   16,  127, 5569,
        7373, 6022, 5474,    1, 5919,  259],
       [8570, 4430, 6413, 6352,   18, 6060, 1631, 2239, 5385, 6696,   26,
        6682, 4506, 7551, 1186, 5448,  122,   14, 2540,    5, 6213,   35,
        9373, 7334, 6409, 2325,   58, 4022, 4555, 1045,    2, 2066, 5329,
        6694, 2898,   22,   17, 3560, 9447, 5563, 3447,   27, 5793, 6859,
        8687, 3590, 8356, 7640, 5744, 3749],
       [4018, 4400, 1412, 2555,    2,  485, 1212, 9517, 2318, 4351,    4,
        4099, 6194, 3907, 9515,    9,   32,   13, 9449,   24, 8912, 3110,
        2989, 7839, 7939, 3519, 2398, 3838, 1378, 1215,   30, 4486, 9085,
        4534, 4454,    8,   24, 5012, 8282, 2649, 9692,   20,  162, 2012,
        8044, 1748,  2

In [25]:
book_tags_df = pd.read_csv("book_tags.csv")

In [26]:
book_tags_df.head(),book_tags_df.shape

(   goodreads_book_id  tag_id   count
 0                  1   30574  167697
 1                  1   11305   37174
 2                  1   11557   34173
 3                  1    8717   12986
 4                  1   33114   12716, (999912, 3))

In [27]:
#a = book_tags_df.loc[book_tags_df["goodreads_book_id"] == 1]

In [28]:
#tags = {"a":1,"s":2,"d":3}
latent_tags = []

In [29]:
for k in range(0,50):
    print("第%d类："%k)
    temdict = {}
    for i in range(0,5):
        user_tags_df = book_tags_df.loc[book_tags_df["goodreads_book_id"] == maxWightBook[i][k]]
        if user_tags_df.empty :
            print("用户%d未打标签!" % int(maxWightBook[i][k]))
        for index,row in user_tags_df.iterrows():
            if row["tag_id"] in temdict.keys():
                temdict[row["tag_id"]] = int(temdict[row["tag_id"]]) + int(row["count"])
            else:
                temdict[row["tag_id"]] = int(row["count"])
    print(temdict)
    temdict = sorted(temdict.items(), key=lambda d:d[1], reverse = True)
    temdict = temdict[:3]
    #print("第%d类："%k)
    print(temdict)
    latent_tags.append(temdict)
    print("tags为：",latent_tags)

第0类：
用户3743未打标签!
用户8570未打标签!
用户4018未打标签!
用户2816未打标签!
用户1793未打标签!
{}
[]
tags为： [[]]
第1类：
用户2775未打标签!
用户4430未打标签!
用户4400未打标签!
用户4062未打标签!
用户7623未打标签!
{}
[]
tags为： [[], []]
第2类：
用户2411未打标签!
用户6413未打标签!
用户1412未打标签!
用户8866未打标签!
用户8050未打标签!
{}
[]
tags为： [[], [], []]
第3类：
用户7237未打标签!
用户6352未打标签!
用户2555未打标签!
用户3064未打标签!
用户6725未打标签!
{}
[]
tags为： [[], [], [], []]
第4类：
用户1205未打标签!
用户18未打标签!
用户4457未打标签!
用户1077未打标签!
{30574: 24549, 8717: 3516, 11305: 3441, 11557: 3423, 6857: 1591, 6888: 1098, 2104: 981, 9221: 975, 11743: 972, 33114: 865, 3389: 851, 5207: 812, 22034: 764, 17124: 756, 22743: 744, 11370: 739, 14017: 734, 11497: 706, 22973: 688, 20849: 621, 33009: 617, 29452: 613, 32623: 566, 31617: 562, 5751: 525, 16427: 519, 24964: 492, 29011: 473, 33012: 464, 6960: 452, 22689: 449, 32989: 448, 20939: 447, 14024: 443, 20824: 438, 18045: 436, 21989: 428, 3371: 426, 33124: 401, 32654: 388, 11491: 380, 26897: 376, 17213: 371, 11590: 365, 33165: 362, 11550: 359, 27535: 344, 16799: 342, 11574: 335, 25150: 

用户94未打标签!
用户32未打标签!
用户4未打标签!
用户14未打标签!
{30574: 29584, 8717: 1345, 11743: 1102, 11557: 843, 14487: 465, 1746: 271, 4949: 213, 7457: 126, 5207: 108, 22743: 105, 11590: 95, 28184: 89, 7852: 86, 14467: 68, 33114: 62, 22034: 62, 5051: 50, 12948: 49, 3389: 46, 18367: 46, 3465: 44, 3358: 42, 26735: 41, 2104: 40, 9886: 37, 14552: 37, 18045: 36, 27535: 36, 1659: 35, 22753: 33, 1642: 33, 2762: 32, 8055: 31, 5857: 31, 7404: 30, 9221: 29, 8067: 29, 4959: 29, 21989: 28, 32989: 27, 30521: 27, 3392: 25, 4985: 25, 27199: 24, 3371: 24, 15626: 21, 4021: 21, 5573: 21, 11505: 20, 20824: 20, 32130: 20, 28422: 20, 11491: 19, 25114: 19, 1416: 19, 14370: 19, 18326: 18, 21689: 18, 1750: 18, 1691: 17, 24840: 17, 11497: 16, 20849: 16, 22689: 15, 12315: 15, 25506: 15, 3467: 15, 20288: 14, 30672: 14, 32586: 14, 26740: 14, 19524: 14, 5481: 14, 24960: 13, 15169: 13, 2106: 13, 9484: 13, 24807: 13, 32917: 13, 8656: 13, 4514: 13, 1128: 12, 10197: 12, 31505: 12, 19733: 12, 4968: 12, 15392: 12, 3466: 12, 2100: 12, 3461: 

{30574: 71925, 11557: 4432, 11305: 2880, 8717: 4466, 11743: 2059, 6953: 1441, 7457: 1347, 33114: 999, 6857: 835, 6888: 747, 6895: 542, 17124: 541, 6828: 512, 32989: 437, 5207: 655, 22743: 675, 19974: 386, 7404: 325, 1691: 400, 6960: 301, 15048: 1025, 6838: 284, 17129: 249, 16799: 214, 6914: 180, 26735: 160, 6920: 149, 24960: 162, 11590: 169, 33165: 138, 2104: 167, 16805: 124, 6976: 114, 18045: 255, 22034: 171, 4949: 498, 6829: 105, 6977: 104, 11505: 113, 25014: 101, 17090: 97, 18367: 170, 22753: 176, 26785: 91, 30521: 147, 12680: 187, 25114: 81, 9221: 174, 25765: 75, 30573: 75, 17213: 174, 19983: 70, 30860: 69, 6847: 67, 20849: 129, 6949: 65, 11497: 63, 6909: 63, 11370: 61, 6597: 61, 6948: 61, 22689: 110, 25034: 60, 6973: 60, 33012: 59, 27535: 69, 17086: 54, 20774: 86, 12315: 53, 26740: 53, 26837: 52, 15169: 84, 6836: 50, 206: 49, 32586: 101, 21892: 48, 12323: 48, 3371: 273, 25020: 46, 30590: 46, 17591: 45, 20824: 43, 33150: 43, 6866: 42, 20785: 42, 19445: 42, 25011: 41, 2277: 113, 696

用户1673未打标签!
用户7235未打标签!
{}
[]
tags为： [[], [], [], [], [(30574, 24549), (8717, 3516), (11305, 3441)], [], [], [(30574, 14770), (13561, 1774), (13547, 968)], [], [], [(30574, 45816), (11557, 40669), (11305, 39687)], [(30574, 3997), (8717, 1638), (24420, 884)], [(30574, 1505), (11743, 626), (11557, 210)], [], [(30574, 12731), (26837, 1535), (26771, 1151)], [(11557, 40087), (11305, 39330), (8717, 17944)], [(30574, 29584), (8717, 1345), (11743, 1102)], [(30574, 110289), (8717, 21347), (11557, 4877)], [(30574, 14018), (7778, 1029), (13561, 866)], [(30574, 48338), (11557, 40684), (11305, 39330)], [(30574, 2949), (11305, 89), (8717, 73)], [(30574, 71925), (8717, 4466), (11557, 4432)], [], [], [(30574, 19313), (11743, 812), (8717, 773)], [], [], [], [], [(30574, 12249), (8717, 1636), (11557, 998)], [(30574, 101111), (8717, 12109), (11557, 6684)], [], []]
第33类：
用户2777未打标签!
用户4534未打标签!
用户2573未打标签!
用户7204未打标签!
{30574: 487, 6953: 322, 6857: 202, 11743: 184, 25992: 177, 11305: 169, 6888: 167, 6828: 

用户127未打标签!
用户5793未打标签!
用户162未打标签!
用户372未打标签!
{30574: 2823, 11557: 144, 11305: 104, 8717: 71, 14017: 54, 33114: 34, 11743: 25, 18886: 23, 22743: 18, 27199: 16, 32989: 15, 5207: 14, 15965: 10, 11590: 8, 29011: 8, 1691: 7, 6857: 7, 6895: 7, 6953: 6, 6828: 6, 27535: 6, 26785: 5, 7457: 5, 11505: 5, 22034: 5, 22973: 5, 32623: 5, 31617: 5, 30521: 5, 2104: 4, 9221: 4, 20774: 4, 29452: 4, 20939: 4, 32654: 4, 18680: 4, 5557: 4, 6888: 3, 15169: 3, 1128: 3, 19974: 3, 16427: 3, 20824: 3, 33165: 3, 18640: 3, 21928: 3, 22545: 3, 3687: 3, 9882: 3, 15048: 3, 14092: 3, 24960: 2, 33009: 2, 10644: 2, 5751: 2, 5090: 2, 14024: 2, 26897: 2, 11491: 2, 11574: 2, 25150: 2, 8055: 2, 1120: 2, 26842: 2, 14008: 2, 14064: 2, 11367: 2, 10731: 2, 2218: 2, 8067: 2, 30358: 2, 32586: 2, 7404: 2, 25506: 2, 26837: 2, 1540: 2, 32996: 2, 11988: 2, 23171: 2, 12680: 2, 14361: 2, 19983: 2, 20547: 2, 24164: 2, 982: 2, 6863: 2, 20828: 2, 32651: 2, 11482: 2, 20818: 2, 32221: 2, 5574: 2, 4375: 2, 4319: 2, 6833: 2, 18890: 2, 33121: 

In [30]:
latent_tags

[[],
 [],
 [],
 [],
 [(30574, 24549), (8717, 3516), (11305, 3441)],
 [],
 [],
 [(30574, 14770), (13561, 1774), (13547, 968)],
 [],
 [],
 [(30574, 45816), (11557, 40669), (11305, 39687)],
 [(30574, 3997), (8717, 1638), (24420, 884)],
 [(30574, 1505), (11743, 626), (11557, 210)],
 [],
 [(30574, 12731), (26837, 1535), (26771, 1151)],
 [(11557, 40087), (11305, 39330), (8717, 17944)],
 [(30574, 29584), (8717, 1345), (11743, 1102)],
 [(30574, 110289), (8717, 21347), (11557, 4877)],
 [(30574, 14018), (7778, 1029), (13561, 866)],
 [(30574, 48338), (11557, 40684), (11305, 39330)],
 [(30574, 2949), (11305, 89), (8717, 73)],
 [(30574, 71925), (8717, 4466), (11557, 4432)],
 [],
 [],
 [(30574, 19313), (11743, 812), (8717, 773)],
 [],
 [],
 [],
 [],
 [(30574, 12249), (8717, 1636), (11557, 998)],
 [(30574, 101111), (8717, 12109), (11557, 6684)],
 [],
 [],
 [(30574, 487), (6953, 322), (6857, 202)],
 [],
 [(30574, 2823), (11557, 144), (11305, 104)],
 [(30574, 36429), (31155, 4503), (21689, 3977)],
 [],

In [31]:
for i in range(0,50):
    if latent_tags[i]:
        latent_tags[i] = [latent_tags[i][0][0],latent_tags[i][1][0],latent_tags[i][2][0]]
    else:
        latent_tags[i] = []

In [32]:
latent_tags

[[],
 [],
 [],
 [],
 [30574, 8717, 11305],
 [],
 [],
 [30574, 13561, 13547],
 [],
 [],
 [30574, 11557, 11305],
 [30574, 8717, 24420],
 [30574, 11743, 11557],
 [],
 [30574, 26837, 26771],
 [11557, 11305, 8717],
 [30574, 8717, 11743],
 [30574, 8717, 11557],
 [30574, 7778, 13561],
 [30574, 11557, 11305],
 [30574, 11305, 8717],
 [30574, 8717, 11557],
 [],
 [],
 [30574, 11743, 8717],
 [],
 [],
 [],
 [],
 [30574, 8717, 11557],
 [30574, 8717, 11557],
 [],
 [],
 [30574, 6953, 6857],
 [],
 [30574, 11557, 11305],
 [30574, 31155, 21689],
 [],
 [30574, 7457, 14487],
 [],
 [30574, 11743, 8717],
 [30574, 8717, 26837],
 [30574, 11557, 11305],
 [],
 [30574, 7457, 8717],
 [30574, 7457, 20939],
 [],
 [30574, 11305, 11557],
 [30574, 8717, 14552],
 []]

In [33]:
tags_df = pd.read_csv("tags.csv")

In [34]:
tags_df.head()

,tag_id,tag_name
0,0,-
1,1,--1-
2,2,--10-
3,3,--12-
4,4,--122-


In [35]:
latentClassExplanation  = []

In [36]:
for i in range(0,50):
    tags_name = []
    if latent_tags[i]:
        for j in range(0,3):
            this_tag_name = tags_df.loc[tags_df["tag_id"] == latent_tags[i][j]]["tag_name"].values[0]
            tags_name.append(this_tag_name)
    latentClassExplanation.append(tags_name)

In [37]:
latentClassExplanation

[[],
 [],
 [],
 [],
 ['to-read', 'currently-reading', 'fantasy'],
 [],
 [],
 ['to-read', 'graphic-novels', 'graphic-novel'],
 [],
 [],
 ['to-read', 'favorites', 'fantasy'],
 ['to-read', 'currently-reading', 'programming'],
 ['to-read', 'fiction', 'favorites'],
 [],
 ['to-read', 'science-fiction', 'sci-fi'],
 ['favorites', 'fantasy', 'currently-reading'],
 ['to-read', 'currently-reading', 'fiction'],
 ['to-read', 'currently-reading', 'favorites'],
 ['to-read', 'comics', 'graphic-novels'],
 ['to-read', 'favorites', 'fantasy'],
 ['to-read', 'fantasy', 'currently-reading'],
 ['to-read', 'currently-reading', 'favorites'],
 [],
 [],
 ['to-read', 'fiction', 'currently-reading'],
 [],
 [],
 [],
 [],
 ['to-read', 'currently-reading', 'favorites'],
 ['to-read', 'currently-reading', 'favorites'],
 [],
 [],
 ['to-read', 'childrens', 'children'],
 [],
 ['to-read', 'favorites', 'fantasy'],
 ['to-read', 'travel', 'non-fiction'],
 [],
 ['to-read', 'classics', 'historical-fiction'],
 [],
 ['to-read', '

In [38]:
for i in range(0,50):
    if latentClassExplanation[i]:
        print("第%d个隐类的标签解释是：%s;%s;%s" % (i,latentClassExplanation[i][0],latentClassExplanation[i][1],latentClassExplanation[i][2]))
    else:
        print("未找到标签解释")

未找到标签解释
未找到标签解释
未找到标签解释
未找到标签解释
第4个隐类的标签解释是：to-read;currently-reading;fantasy
未找到标签解释
未找到标签解释
第7个隐类的标签解释是：to-read;graphic-novels;graphic-novel
未找到标签解释
未找到标签解释
第10个隐类的标签解释是：to-read;favorites;fantasy
第11个隐类的标签解释是：to-read;currently-reading;programming
第12个隐类的标签解释是：to-read;fiction;favorites
未找到标签解释
第14个隐类的标签解释是：to-read;science-fiction;sci-fi
第15个隐类的标签解释是：favorites;fantasy;currently-reading
第16个隐类的标签解释是：to-read;currently-reading;fiction
第17个隐类的标签解释是：to-read;currently-reading;favorites
第18个隐类的标签解释是：to-read;comics;graphic-novels
第19个隐类的标签解释是：to-read;favorites;fantasy
第20个隐类的标签解释是：to-read;fantasy;currently-reading
第21个隐类的标签解释是：to-read;currently-reading;favorites
未找到标签解释
未找到标签解释
第24个隐类的标签解释是：to-read;fiction;currently-reading
未找到标签解释
未找到标签解释
未找到标签解释
未找到标签解释
第29个隐类的标签解释是：to-read;currently-reading;favorites
第30个隐类的标签解释是：to-read;currently-reading;favorites
未找到标签解释
未找到标签解释
第33个隐类的标签解释是：to-read;childrens;children
未找到标签解释
第35个隐类的标签解释是：to-read;favorites;fantasy
第36个隐类的标签解释是：to-read;travel;non-fiction
未